## Renaming
- IPC522-time-random_num-preset-weather-image_num

In [27]:
import urllib.request
import re

def getWeather(date, stn = "112"):
    year = date[:4]
    mm = date[4:6]
    dd = date[6:]
    # print(year, mm, dd)
    # url = "https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=" + stn + "&yy=" + year + "&mm=" + mm + "&obs=1"
    url = "https://web.kma.go.kr/weather/climate/past_cal.jsp?stn=" + stn + "&yy=" + year + "&mm=" + mm + "&obs=1&x=24&y=9"
    
# https://www.weather.go.kr/weather/climate/past_cal.jsp?stn=112&yy=2021&mm=07&obs=1&x=24&y=9 ##2107
    lines = []

    f = urllib.request.urlopen(url)
    r = f.read()
    f.close()
    r2 = r.decode('euc-kr', 'ignore')

    lines = r2.split('\n')

    regex = '.*<td class="align_left">평균기온:(.*?)<br \/>최고기온:(.*?)<br \/>최저기온:(.*?)<br \/>평균운량:(.*?)<br \/>일강수량:(.*?)<br \/><\/td>'

    dict_month = {}
    day = 1
    dd = int(dd)

    for l in lines:
        if not '평균기온' in l: continue

        l = l.replace("℃", "")

        l_reg = re.match(regex, l)
        if not l_reg: continue

        dict_day = {'cloud':0, 'rain':0}

        data_cloud = l_reg.groups()[3]   # 평균운량
        data_rain = l_reg.groups()[4]    # 일강수량

        dict_day['cloud'] = data_cloud   # 평균운량
        dict_day['rain'] = data_rain.replace("-", "0").replace("mm", "")    # 일강수량
        if day == dd:
            dict_month[dd] = dict_day
        day = day + 1

    for (day, dict_day) in dict_month.items():
        # print ("{0}{1}{2}, cloud : {3}, rain : {4} ".format(year, mm.zfill(2), str(day).zfill(2), dict_day['cloud'], dict_day['rain']))
        _date = year + mm.zfill(2) + str(day).zfill(2)
        # print(mm)
        # print(day)
        # print(_date)
        _cloud = dict_day['cloud']
        _rain = dict_day['rain']
    return _date, _cloud, _rain

In [28]:
def get_weather(time):
    date, cloud, rain = getWeather(time)

    if float(cloud) <= 5. and float(rain) == 0.:
        weather = 'sunny'
    elif float(rain) != 0.:
        weather = 'rainy'
    elif float(cloud) > 5. and float(rain) == 0.:
        weather = 'foggy'
            
    return weather

In [33]:
import os
import glob

import pandas as pd
import numpy as np

In [34]:
filename = 'wrong_preset_in_ours'
df = pd.read_excel(f'before/{filename}.xlsx')
df.head()

,before,preset
0,IPC522_20210616153627_133_p08_sunny_00015.jpg,1
1,IPC522_20210818092656_033_p02_rainy_00080.jpg,3
2,IPC522_20210818092656_033_p02_rainy_00081.jpg,3
3,IPC522_20210818092656_033_p02_rainy_00082.jpg,3
4,IPC522_20210818092656_033_p02_rainy_00083.jpg,3


In [36]:
df['time'] = df['before'].apply(lambda x: x.split('_')[1])

In [37]:
df['random_num'] = -1
df['weather'] = -1
df['image_num'] = -1
df['new_filename'] = -1

# df = df[['before', 'start_point', 'time', 'random_num', 'preset', 'weather', 'image_num', 'new_filename']]
df = df[['before', 'time', 'random_num', 'preset', 'weather', 'image_num', 'new_filename']]
df.head()

,before,time,random_num,preset,weather,image_num,new_filename
0,IPC522_20210616153627_133_p08_sunny_00015.jpg,20210616153627,-1,1,-1,-1,-1
1,IPC522_20210818092656_033_p02_rainy_00080.jpg,20210818092656,-1,3,-1,-1,-1
2,IPC522_20210818092656_033_p02_rainy_00081.jpg,20210818092656,-1,3,-1,-1,-1
3,IPC522_20210818092656_033_p02_rainy_00082.jpg,20210818092656,-1,3,-1,-1,-1
4,IPC522_20210818092656_033_p02_rainy_00083.jpg,20210818092656,-1,3,-1,-1,-1


- IPC522-time-random_num-preset-weather-image_num

In [38]:
id_list = df['time'].unique().tolist()

for i, time in enumerate(id_list):
    data_len = len(df.loc[df['time'] == time])

    df.loc[df['time'] == time, 'random_num'] = str(i).zfill(3)
    df.loc[df['time'] == time, 'weather'] = get_weather(str(time)[:8])
    df.loc[df['time'] == time, 'image_num'] = range(data_len)
    df.loc[df['time'] == time, 'image_num'] = df.loc[df['time'] == time, 'image_num'].astype(str).str.zfill(5)
    df.loc[df['time'] == time, 'preset'] = df.loc[df['time'] == time, 'preset'].astype(str).str.zfill(2)
    df.loc[df['time'] == time, 'preset'] = df.loc[df['time'] == time, 'preset'].astype(str).apply(lambda x: 'p'+x)
    
df

,before,time,random_num,preset,weather,image_num,new_filename
0,IPC522_20210616153627_133_p08_sunny_00015.jpg,20210616153627,000,p01,sunny,00000,-1
1,IPC522_20210818092656_033_p02_rainy_00080.jpg,20210818092656,001,p03,rainy,00000,-1
2,IPC522_20210818092656_033_p02_rainy_00081.jpg,20210818092656,001,p03,rainy,00001,-1
3,IPC522_20210818092656_033_p02_rainy_00082.jpg,20210818092656,001,p03,rainy,00002,-1
4,IPC522_20210818092656_033_p02_rainy_00083.jpg,20210818092656,001,p03,rainy,00003,-1
...,...,...,...,...,...,...,...
67,IPC522_20210818092656_033_p02_rainy_00146.jpg,20210818092656,001,p03,rainy,00066,-1
68,IPC522_20210818092656_033_p02_rainy_00147.jpg,20210818092656,001,p03,rainy,00067,-1
69,IPC522_20210818092656_033_p02_rainy_00157.jpg,20210818092656,001,p03,rainy,00068,-1
70,IPC522_20210818092656_033_p02_rainy_00158.jpg,20210818092656,001,p03,rainy,00069,-1


In [ ]:
cols = ['time', 'random_num', 'preset', 'weather', 'image_num']
df['new_filename'] = df[cols].apply(lambda x: '_'.join(x.values.astype(str)), axis=1)
df['new_filename'] = 'IPC522_' + df['new_filename'].astype(str) + '.jpg'
df.drop(['random_num', 'weather', 'image_num'], inplace=True, axis=1)
df

In [76]:
df.to_csv(f'after/{filename}_renamed.csv', index=False)
df

,before,time,random_num,preset,weather,image_num,new_filename,target_filename,filename
0,IPC522_20210616153627_133_p08_sunny_00015.jpg,20210616153627,000,p01,sunny,00000,IPC522_20210616153627_133_p01_sunny_00015.jpg,S:/public_data/segmentation/second_inspector_r...,S:/public_data/segmentation/wrong_preset_in_ou...
1,IPC522_20210818092656_033_p02_rainy_00080.jpg,20210818092656,001,p03,rainy,00000,IPC522_20210818092656_033_p03_rainy_00080.jpg,S:/public_data/segmentation/second_inspector_r...,S:/public_data/segmentation/wrong_preset_in_ou...
2,IPC522_20210818092656_033_p02_rainy_00081.jpg,20210818092656,001,p03,rainy,00001,IPC522_20210818092656_033_p03_rainy_00081.jpg,S:/public_data/segmentation/second_inspector_r...,S:/public_data/segmentation/wrong_preset_in_ou...
3,IPC522_20210818092656_033_p02_rainy_00082.jpg,20210818092656,001,p03,rainy,00002,IPC522_20210818092656_033_p03_rainy_00082.jpg,S:/public_data/segmentation/second_inspector_r...,S:/public_data/segmentation/wrong_preset_in_ou...
4,IPC522_20210818092656_033_p02_rainy_00083.jpg,20210818092656,001,p03,rainy,00003,IPC522_20210818092656_033_p03_rainy_00083.jpg,S:/public_data/segmentation/second_inspector_r...,S:/public_data/segmentation/wrong_preset_in_ou...
...,...,...,...,...,...,...,...,...,...
67,IPC522_20210818092656_033_p02_rainy_00146.jpg,20210818092656,001,p03,rainy,00066,IPC522_20210818092656_033_p03_rainy_00146.jpg,S:/public_data/segmentation/second_inspector_r...,S:/public_data/segmentation/wrong_preset_in_ou...
68,IPC522_20210818092656_033_p02_rainy_00147.jpg,20210818092656,001,p03,rainy,00067,IPC522_20210818092656_033_p03_rainy_00147.jpg,S:/public_data/segmentation/second_inspector_r...,S:/public_data/segmentation/wrong_preset_in_ou...
69,IPC522_20210818092656_033_p02_rainy_00157.jpg,20210818092656,001,p03,rainy,00068,IPC522_20210818092656_033_p03_rainy_00157.jpg,S:/public_data/segmentation/second_inspector_r...,S:/public_data/segmentation/wrong_preset_in_ou...
70,IPC522_20210818092656_033_p02_rainy_00158.jpg,20210818092656,001,p03,rainy,00069,IPC522_20210818092656_033_p03_rainy_00158.jpg,S:/public_data/segmentation/second_inspector_r...,S:/public_data/segmentation/wrong_preset_in_ou...


### filename 바꾸기

In [64]:
import os
import glob
import shutil

import pandas as pd

In [3]:
renamed_list = os.listdir('after')

In [59]:
df_renamed = pd.concat([df_renamed1, df_renamed2])
df_renamed

,before,time,preset,new_filename
0,preset_8_20210527_114147.jpeg,20210527114146,p08,IPC522_20210527114146_000_p08_rainy_00000.jpg
1,preset_8_20210527_115348.jpeg,20210527114146,p08,IPC522_20210527114146_000_p08_rainy_00001.jpg
2,preset_8_20210527_121350.jpeg,20210527114146,p08,IPC522_20210527114146_000_p08_rainy_00002.jpg
3,preset_8_20210527_122150.jpeg,20210527114146,p08,IPC522_20210527114146_000_p08_rainy_00003.jpg
4,preset_8_20210527_122551.jpeg,20210527114146,p08,IPC522_20210527114146_000_p08_rainy_00004.jpg
...,...,...,...,...
511,preset_9_20210603_150900.jpeg,20210603140053,p01,IPC522_20210603140053_041_p01_rainy_00016.jpg
512,preset_9_20210603_151301.jpeg,20210603140053,p01,IPC522_20210603140053_041_p01_rainy_00017.jpg
513,preset_9_20210603_151701.jpeg,20210603140053,p01,IPC522_20210603140053_041_p01_rainy_00018.jpg
514,preset_9_20210603_152101.jpeg,20210603140053,p01,IPC522_20210603140053_041_p01_rainy_00019.jpg


In [65]:
base_url = 'S:/public_data/segmentation/wrong_preset_in_ours/'
target_url = 'S:/public_data/segmentation/second_inspector_renamed/'
image_list = glob.glob(base_url + '/*')
basename_list = [os.path.basename(img) for img in image_list]

len(basename_list)

72

In [68]:
df_renamed['target_filename'] = df_renamed['new_filename'].apply(lambda x: target_url+x)
df_renamed['filename'] = df_renamed.before.apply(lambda x: base_url+x)
df_renamed

,before,time,random_num,preset,weather,image_num,new_filename,target_filename,filename
0,IPC522_20210616153627_133_p08_sunny_00015.jpg,20210616153627,000,p01,sunny,00000,IPC522_20210616153627_133_p01_sunny_00015.jpg,S:/public_data/segmentation/second_inspector_r...,S:/public_data/segmentation/wrong_preset_in_ou...
1,IPC522_20210818092656_033_p02_rainy_00080.jpg,20210818092656,001,p03,rainy,00000,IPC522_20210818092656_033_p03_rainy_00080.jpg,S:/public_data/segmentation/second_inspector_r...,S:/public_data/segmentation/wrong_preset_in_ou...
2,IPC522_20210818092656_033_p02_rainy_00081.jpg,20210818092656,001,p03,rainy,00001,IPC522_20210818092656_033_p03_rainy_00081.jpg,S:/public_data/segmentation/second_inspector_r...,S:/public_data/segmentation/wrong_preset_in_ou...
3,IPC522_20210818092656_033_p02_rainy_00082.jpg,20210818092656,001,p03,rainy,00002,IPC522_20210818092656_033_p03_rainy_00082.jpg,S:/public_data/segmentation/second_inspector_r...,S:/public_data/segmentation/wrong_preset_in_ou...
4,IPC522_20210818092656_033_p02_rainy_00083.jpg,20210818092656,001,p03,rainy,00003,IPC522_20210818092656_033_p03_rainy_00083.jpg,S:/public_data/segmentation/second_inspector_r...,S:/public_data/segmentation/wrong_preset_in_ou...
...,...,...,...,...,...,...,...,...,...
67,IPC522_20210818092656_033_p02_rainy_00146.jpg,20210818092656,001,p03,rainy,00066,IPC522_20210818092656_033_p03_rainy_00146.jpg,S:/public_data/segmentation/second_inspector_r...,S:/public_data/segmentation/wrong_preset_in_ou...
68,IPC522_20210818092656_033_p02_rainy_00147.jpg,20210818092656,001,p03,rainy,00067,IPC522_20210818092656_033_p03_rainy_00147.jpg,S:/public_data/segmentation/second_inspector_r...,S:/public_data/segmentation/wrong_preset_in_ou...
69,IPC522_20210818092656_033_p02_rainy_00157.jpg,20210818092656,001,p03,rainy,00068,IPC522_20210818092656_033_p03_rainy_00157.jpg,S:/public_data/segmentation/second_inspector_r...,S:/public_data/segmentation/wrong_preset_in_ou...
70,IPC522_20210818092656_033_p02_rainy_00158.jpg,20210818092656,001,p03,rainy,00069,IPC522_20210818092656_033_p03_rainy_00158.jpg,S:/public_data/segmentation/second_inspector_r...,S:/public_data/segmentation/wrong_preset_in_ou...


In [73]:
df_renamed['target_filename'].iloc[4]

'S:/public_data/segmentation/second_inspector_renamed/IPC522_20210818092656_033_p03_rainy_00083.jpg'

In [74]:
for before, after in zip(df_renamed['filename'], df_renamed['target_filename']):
    shutil.copy(before, after)